### __Introducción__

Una de las librerías más populares es notion-client, la cual proporciona una interfaz para interactuar con la API oficial de Notion. Esta librería te permite realizar diversas operaciones como leer, crear, actualizar y eliminar páginas, bases de datos y bloques dentro de Notion.

In [115]:
from notion_client import Client

##### Crear el cliente con el token de integración

Para utilizar la API de Notion, se necesita autenticación usando un token de integración que se obtiene al configurar la integración en Notion.

In [116]:
Notion = Client(auth="secret_NYOvwjAc63k8bhNv0zpfaeUX1S7dk5ilgek1hoWXKkR")

##### Establecer el ID de la página o bloque de Notion

In [117]:
ID_Database = "129a53d444454d63820814429c4fb691"
ID_Page = '3b753b78-187f-414f-a78c-a9d106cca7fb'

##### Obtener estructura de una base de datos

Obtener la estructura de la base de datos para ver las propiedades con el método __databases.retrieve().__

In [118]:
Database = Notion.databases.retrieve(database_id=ID_Database)

##### Filtrar y ordenar base de datos

La función __databases.query__ permite consultar una base de datos específica de Notion usando su database_id. Se puede aplicar filtros, ordenar y paginar los resultados.

__Parámetros.__

- _database_id_. El id de la base de datos que se va a leer.
- _filter._ Filtra resultados basados en propiedades específicas. Dentro, podés especificar qué contiene con _contains: Valor_. Hay varios tipos de propiedades:
    - _title_. Título.
    - _rich_ _ _text_. Texto.
    - _number_. Número.
    - _select_. Permite seleccionar un valor único de una lista predefinida de opciones.
    - _multi_ _ _select_. Permite seleccionar múltiples opciones de una lista predefinida.
    - _date_. Fecha.
- _sorts_. Cómo se ordenan los datos.
    - _property._ Debe coincidir exactamente con el nombre tal como aparece en las propiedades del objeto de la base de datos.
    - _direction._ Se puede usar "ascending" o "descending" para ordenar los resultados.

In [119]:
Filtro = Notion.databases.query(
    **{
        "database_id": ID_Database,
        "filter": {
            "property": "Name",
            "title": {
                "contains": "Séneca"
            }
        },
        "sorts": [
            {
                "property": "Last edited time",  
                "direction": "descending"
            }
        ]
    }
)


# Accede a los resultados.
for Elementos in Filtro['results']:
    print(Elementos)

# Filtra los elementos que contienen 'Seneca' en la propiedad 'Name'.
# Los ordena por el último edit en orden descendente.

{'object': 'page', 'id': '3b753b78-187f-414f-a78c-a9d106cca7fb', 'created_time': '2021-08-23T15:22:00.000Z', 'last_edited_time': '2024-09-09T23:09:00.000Z', 'created_by': {'object': 'user', 'id': 'a098d629-3725-4c73-b9b5-119ecbc0e26e'}, 'last_edited_by': {'object': 'user', 'id': 'a098d629-3725-4c73-b9b5-119ecbc0e26e'}, 'cover': {'type': 'external', 'external': {'url': 'https://static1.abc.es/media/cultura/2018/11/04/Seneca-kYAD--1248x698@abc.jpg'}}, 'icon': {'type': 'emoji', 'emoji': '🎗️'}, 'parent': {'type': 'database_id', 'database_id': '129a53d4-4445-4d63-8208-14429c4fb691'}, 'archived': False, 'in_trash': False, 'properties': {'Last edited time': {'id': 'O%60g%40', 'type': 'last_edited_time', 'last_edited_time': '2024-09-09T23:09:00.000Z'}, 'Name': {'id': 'title', 'type': 'title', 'title': [{'type': 'text', 'text': {'content': 'Séneca', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'pla

### __Páginas__

##### Obtener una página

La función __pages.retrieve__ obtiene los detalles de una página específica usando su page_id. Se utiliza para obtener información detallada sobre una página, incluyendo propiedades y contenido.

In [120]:
Page = Notion.pages.retrieve(page_id=ID_Page)

##### Crear una página

La función __pages.create__ permite crear una nueva página en una base de datos o en un espacio de trabajo.

__Parámetros.__

- _parent._ La database padre donde va a ser construida.
- _name._ Su nombre.

In [121]:
New_Page = Notion.pages.create(
    **{
        "parent": {"database_id": ID_Database},
        "properties": {
            "Name": {
                "title": [
                    {
                        "text": {
                            "content": "Página nueva"
                        }
                    }
                ]
            }
        }
    }    
)


##### Actualizar una página

Actualiza las propiedades de una página existente.

In [122]:
Pagina_Nueva = 'f138c17b4dd44eebb3e8d7ec1ef2b724'

# Actualiza el título de la página.
Pagina_Actualizada = Notion.pages.update(
    page_id=Pagina_Nueva,
    properties = {
        "Name": {
            "title": [
                {
                    "text": {
                        "content": "Página chota"
                    }
                }
            ]
        }
    }    
)

### __Bloques__

##### Agregar contenido a un bloque o página

La función __block.children.append()__ agrega contenido a un bloque de una página, como texto, imágenes, listas, etc.

In [123]:
Bloque_Nuevo = Notion.blocks.children.append(
    block_id=Pagina_Nueva,
    children=[
        {
            "object": "block",
            "type": "paragraph",
            "paragraph": {
                "rich_text": [
                    {
                        "type": "text",
                        "text": {
                            "content": "Este es un nuevo párrafo en la página."
                        }
                    }
                ]
            }
        }
    ]
)

# Agrega un texto a una página.

##### Obtener los ID de todos los bloques de una página

El método __blocks.children.list()__ permite listar todos los bloques dentro de una página.

In [124]:
# Se obtiene la información de todos los bloques hijos de la página.
Resultado = Notion.blocks.children.list(block_id=Pagina_Nueva)

# Se extrae los IDs de los bloques de la información generada.
Blocks_IDs = [Block['id'] for Block in Resultado['results']]

Si hay muchos bloques, la información generada podría estar paginada. En ese caso, se usa next_cursor para solicitar la siguiente página de los resultados. 

In [1]:
while Has_More:

    # Solicita los bloques, incluyendo el cursor si existe
    Resultado = Notion.blocks.children.list(block_id=Pagina_Nueva, start_cursor=Cursor)
    
    # Añade los IDs de los bloques a la lista
    Blocks_IDs.extend([Block['id'] for Block in Resultado['results']])
    
    # Verifica si hay más resultados y actualiza el cursor.
    Has_More = Resultado['has_more']
    Cursor = Resultado['next_cursor']

- _start_cursor=cursor_ permite solicitar la siguiente página de bloques.

##### Editar el contenido de un bloque

Con el método blocks.update se puede actualizar la información de un bloque.

In [126]:
# Bloque a editar.
ID_Bloque = Blocks_IDs[0]

In [127]:
# Datos para actualizar el bloque, por ejemplo, un bloque de párrafo con nuevo texto.
Data_Actualizada = {
    "paragraph": {
        "rich_text": [
            {
                "type": "text",
                "text": {
                    "content": "Este es el nuevo contenido del bloque."
                }
            }
        ]
    }
}

In [128]:
# Actualiza la información del bloque.
Response = Notion.blocks.update(block_id=ID_Bloque, **Data_Actualizada)

##### Editar el tipo de bloque

La API de Notion no permite cambiar directamente ciertos tipos de bloques a otros, como convertir un párrafo en un callout o viceversa.

Para solucionar esto, se siguen estos pasos:

- Guardar contenido del bloque original.
- Crear un nuevo bloque con los datos guardados en la misma posición.
- Eliminar bloque original.
- Insertar nuevo bloque.

In [129]:
# Paso 1: Obtener los detalles del bloque existente sabiendo que es 'paragraph'.

Block = Notion.blocks.retrieve(block_id=ID_Bloque)
Block_Content = Block['paragraph']['rich_text'][0]['text']['content']

In [130]:
# Paso 2: Crear un nuevo callout justo después del bloque original.

Bloque_Nuevo = {
    "object": "block",
    "type": "callout",
    "callout": {
        "rich_text": [
            {
                "type": "text",
                "text": {
                    "content": Block_Content
                }
            }
        ],
        "icon": {
            "type": "emoji",
            "emoji": "💡"
        }
    }
}

In [131]:
# Paso 3: Inserta el nuevo bloque.
Insertar = Notion.blocks.children.append(
    block_id=Block['parent']['page_id'],
    children=[Bloque_Nuevo]
)

In [132]:
# Paso 4: Eliminar el bloque original.
Eliminar = Notion.blocks.delete(block_id=ID_Bloque)